<a href="https://colab.research.google.com/github/manjotmb20/EC3/blob/master/bgcm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Implementing the BGCM algorithm**

**downloading libraries**

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from numpy import linalg as LA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import StandardScaler
import nltk
nltk.download("punkt")
# Warnings
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Data preprocessing on the dataset**

In [0]:
df_train = pd.read_csv('train.csv')
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)
df_train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)
df_train[['SibSp', 'Survived']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)
df_train[['Parch', 'Survived']].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)
df_train = df_train.drop(['Ticket', 'Cabin'], axis=1)
combine = [df_train]
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract('([A-Za-z]+)\.', expand=False)
pd.crosstab(df_train['Title'], df_train['Sex'])
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Don', 'Sir', 'Jonkheer', 'Dona'],'Royalty')
    dataset['Title'] = dataset['Title'].replace(['Capt', 'Col','Dr','Major','Rev'],'Special')
df_train[['Title','Survived']].groupby(['Title'], as_index=False).mean()
title_mapping = {"Master": 1, "Miss": 2, "Mrs": 3, "Mr": 4, "Royalty": 5, "Special": 6}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
df_train = df_train.drop(['Name', 'PassengerId'], axis=1)
combine = [df_train]
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
median_age = np.zeros((2,3))
for dataset in combine:
    for sex in range(0,2):
        for pclass in range(0,3):
            guess_df = dataset[(dataset['Sex'] == sex) & \
                                  (dataset['Pclass'] == pclass+1)]['Age'].dropna()
            age_guess = guess_df.median()
            median_age[sex,pclass] = age_guess

for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = median_age[i,j]

dataset['Age'] = dataset['Age'].astype(int)
df_train['AgeBand'] = pd.cut(df_train['Age'], 5)
df_train[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)
for dataset in combine:
    dataset.loc[ dataset['Age'] <= 16, 'AgeG'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'AgeG'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'AgeG'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'AgeG'] = 3
    dataset.loc[ dataset['Age'] > 64, 'AgeG'] = 4
df_train = df_train.drop(['AgeBand','Age'], axis=1)
combine = [df_train]
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

df_train[['FamilySize','Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False) 
df_train = df_train.drop(['SibSp', 'Parch'], axis=1)
combine = [df_train]
freq_port = df_train.Embarked.dropna().mode()[0]
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
df_train[['Embarked','Survived']].groupby(['Embarked'], as_index = False).mean().sort_values(by='Survived', ascending=False)
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
df_train = df_train.drop(['Fare'], axis=1)
combine = [df_train]
for dataset in combine:
    dataset.AgeG = dataset.AgeG.astype(int)
X_train = df_train.drop("Survived", axis=1)
Y_train = df_train['Survived'] 

**Applying classification algorithm on the dataset to form the output used by the BGCM algorithm.**

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
classifier3=SVC(probability=True)
from sklearn.metrics import roc_auc_score,accuracy_score
x_train,x_val,y_train,y_val=train_test_split(X_train,Y_train,test_size=0.4,random_state=2)
classifier=LogisticRegression(random_state=0)
classifier.fit(x_train,y_train)
ypred=classifier.predict(x_val)
print("ROC score of LR",roc_auc_score(y_val,ypred))
y_prob=classifier.predict_proba(x_val)
df=pd.DataFrame()
df['pred']=ypred
survived=pd.get_dummies(df.pred,prefix='Survived')
df=pd.concat([df,survived],axis=1)
df.drop('pred',inplace=True,axis=1)
df2=pd.DataFrame()
classifier3.fit(x_train,y_train)
ypred3=classifier3.predict(x_val)
print("ROC score of SVM",roc_auc_score(y_val,ypred3))
df2['pred']=ypred3
y_prob3=classifier3.predict_proba(x_val)
survived2=pd.get_dummies(df2.pred,prefix='Surviveds')
df=pd.concat([df,survived2],axis=1)

ROC score of LR 0.8043450148713306
ROC score of SVM 0.8082891503944135


In [0]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2,max_iter=100)
kmeans.fit(x_train)
k_pred=kmeans.predict(x_val)
accuracy_score(y_val,k_pred)
df_kmeans=pd.DataFrame()
df_kmeans['Survived_k']=k_pred
survivedk=pd.get_dummies(df_kmeans.Survived_k,prefix='Survivedk')


In [23]:
df['Survivedk_0']=survivedk['Survivedk_0']
df['Survivedk_1']=survivedk['Survivedk_1']
print("Accuracy score of Kmeans",accuracy_score(y_val,k_pred))
df

Accuracy score of Kmeans 0.5938375350140056


,Survived_0,Survived_1,Surviveds_0,Surviveds_1,Survivedk_0,Survivedk_1
0,1,0,1,0,1,0
1,1,0,1,0,1,0
2,0,1,0,1,0,1
3,1,0,1,0,1,0
4,1,0,1,0,0,1
...,...,...,...,...,...,...
352,1,0,1,0,1,0
353,0,1,0,1,1,0
354,0,1,0,1,1,0
355,1,0,1,0,1,0


**Generating affinity matrix dimension (n*v), here v=4**

In [24]:
A=np.array(df)
A.shape

(357, 6)

**Generating conditional probability  matrix U of dimension** **(n*c)**

In [0]:

objclass_mat1=pd.DataFrame(y_prob,columns=['Survived','NotSurvived'])
objclass_mat2=pd.DataFrame(y_prob3,columns=['Survived','NotSurvived'])
df_t=pd.DataFrame(objclass_mat1)
df_t['Survived2']=objclass_mat2['Survived']
df_t['NotSurvived2']=objclass_mat2['NotSurvived']


G=4
C=2
m=G/2
n=df_t.shape[0]
dft=np.array(df_t)
Fo=np.zeros(shape=(n,C))
for i in range(n):
  sum1=0
  for j in range(0,G,2):
    sum1=sum1+dft[i][j]
  Fo[i][0]=sum1/m 
  sum2=0
  for j in range(1,G,2):
    sum2=sum2+dft[i][j]
  Fo[i][1]=sum2/m
U=np.array(Fo)

**Checking for Fo conditions.**

In [27]:
np.linalg.norm((Fo[0]),ord=1)

1.0

In [28]:
Fo.shape

(357, 2)

**Similarly calculating Q and Y as per the BGCM algorithm**

In [29]:
m=np.array(df)
Yg=np.zeros(shape=(m.shape[1],2))
c=np.sum(m,axis=0)
for i in range(Yg.shape[0]):
  if i%2==0:
    Yg[i][0]=c[i]/m.shape[0]
    Yg[i][1]=1-Yg[i][0]
  else:
    Yg[i][1]=c[i]/m.shape[0]
    Yg[i][0]=1-Yg[i][1]  
print(Yg.shape)    
Fg=Yg
Q=np.array(Fg)
Y=np.array(Yg)
for i in range(Y.shape[0]):
  for j in range(Y.shape[1]):
    if Y[i][j]>0.5:
      Y[i][j]=1
    else:
      Y[i][j]=0

(6, 2)


**Importing and defining epsilon**

In [0]:
import sys
threshold=sys.float_info.epsilon

**Generating diagonal matrices Dv, Dn and K respectively**

In [0]:
D_sum=np.sum(A,axis=0)
Dv=np.diag(D_sum)
Dn_sum=np.sum(A,axis=1)
Dn=np.diag(Dn_sum)
K_sum=np.sum(Y,axis=1)
K=np.diag(K_sum)


**Implementing BGCM algorithm as per the paper**

In [0]:
Ut_old=np.zeros(shape=U.shape)
alpha=0.40
Ut=U
while LA.norm((Ut-Ut_old))>threshold:
  Ut_old=Ut
  Q=np.matmul((np.linalg.inv(Dv+alpha*K)),((np.matmul(np.transpose(A),Ut))+(alpha*np.matmul(K,Y))))
  t=np.matmul(A,Q)
  Ut=np.matmul(np.linalg.inv(Dn),t)
Ut.shape  
from sklearn.metrics import accuracy_score
result=np.zeros(shape=Ut.shape[0])
for i in range(Ut.shape[0]):
  if Ut[i][0]<Ut[i][1]:
    result[i]=1